In [ ]:
library(Seurat)
library(Signac)
library(GenomeInfoDb)
library(EnsDb.Mmusculus.v79)
library(ggplot2)
library(ggpubr)
library(glue)
library(scDblFinder)
library(RColorBrewer)
library(dplyr)
library(ggridges)
library(CopyscAT)
library(BSgenome.Mmusculus.UCSC.mm10)
library(harmony)
library(RColorBrewer)
library(data.table)
library(patchwork)
library(RColorBrewer)
set.seed(123)


## merge ATAC and GEX data by shared cells

In [2]:
atac_obj = readRDS("atac_subGex_clustered_obj.RDS")


In [3]:
gex_obj = readRDS("./gex/GEX_cDC1_clustered.RDS")

In [ ]:
head(colnames(atac_obj))

In [ ]:
head(gex_obj$atac_cellID)

In [10]:
colnames(gex_obj) = gex_obj$atac_cellID

In [12]:
common.cells <- intersect(colnames(gex_obj), colnames(atac_obj))
gex <- subset(gex_obj, cells = common.cells)


In [ ]:
dim(gex)
dim(atac_obj)

In [17]:
# check if the colnames are the same
setequal(colnames(gex), colnames(atac_obj))


[1] TRUE

In [18]:
# Extract ATAC assay and add it to the GEX object
atac_assay <- atac_obj[["peaks"]]  
gex[["peaks"]] <- atac_assay

In [30]:
atac_meta = atac_obj@meta.data
gex_meta = gex@meta.data

In [35]:
atac_meta = atac_meta[rownames(gex_meta), ]

In [ ]:
identical(rownames(atac_meta), rownames(gex@meta.data))  


[1] TRUE

In [37]:
gex@meta.data <- cbind(gex@meta.data, atac_meta)

In [40]:
saveRDS(gex, "atac_gex_combined_obj.RDS")

# Linking peaks to genes of interest (ccr7, Il12b, cd80, cd86)

In [ ]:
DefaultAssay(gex) <- "peaks"
gex <- RegionStats(gex, genome = BSgenome.Mmusculus.UCSC.mm10)

# link peaks to genes
gex <- LinkPeaks(
  object = gex,
  peak.assay = "peaks",
  expression.assay = "RNA",
  genes.use = c("Ccr7", "Il12b", "Cd80", "Cd86")
)

In [52]:
saveRDS(gex, "atac_gex_combined_obj.RDS")